# Sources:

[1] ["Who’s At The Keyboard? Authorship Attribution in Digital Evidence Investigations"](www.utica.edu/academic/institutes/ecii/publications/articles/B49F9C4A-0362-765C-6A235CB8ABDFACFF.pdf): good reference regarding number of authors and text volume.

[2] ["How a Computer Program Helped Show J.K. Rowling write A Cuckoo’s Calling"](https://www.scientificamerican.com/article/how-a-computer-program-helped-show-jk-rowling-write-a-cuckoos-calling/): some good ideas for feature to analyse.


# Research Notes:

- test approaches suggested in [1]
- measure average word lentgh per article, distribution of word lengths, the 100 most common words, distribution of character 4-grams, word bigrams
- test with support vector machines
- using word embeddings as a much rich representation




# Introduction



# Data Acquisition

--a bit of background about the approach, why only Medium, something with a uniform structure that simplifies scraping without much post processing--

-- mention that we will only describe briefly what the scrape script does, nos discussion of code in detail as this is no the main topic of this capstone project --

For this project we will use the self publishing platform Medium as a source for sample authors and their articles. Conveniently the website [Top Authors](https://topauthors.xyz/) has published a list of 300+ top Medium publishers. The project folder contains the short script `get_list_of_Medium_authors.py` that has been used to extract the URLs for each author.

The initial list of 300+ authors has been reduced to 25 (the Medium URLs for these authors can be found in file `Medium_authors_25.txt`). The criteria for this selection was the number of published articles. For the 25 authors there are at least 300 articles available.

The actual collection of the articles is done with script `pull_Medium_articles.py`. The script performs two steps. First, it builds a list of all articles URLs and for each article saves author URL and article URL in JSON format in file `Medium_article_urls.json`. Below is an example how this looks like.

```javascript
{"author_URL": "https://medium.com/@tedr/latest\n",
 "article_URL": "https://medium.com/season-of-the-witch/etiquette-and-the-cancer-patient-630a50047448?source=user_profile---------1----------------"}
{"author_URL": "https://medium.com/@esterbloom/latest\n",
 "article_URL": "https://medium.com/the-billfold/failing-at-shoplifting-life-with-kesha-bc2600b1f440?source=user_profile---------789----------------"}
{"author_URL": "https://medium.com/@gassee/latest",
 "article_URL": "https://mondaynote.com/the-ny-times-un-free-at-last-df2eddba360b?source=user_profile---------281----------------"}
```

Second, ...



-- list some of the challenges of scraping, speed, scrapy didn't work on dynamic content websites, authentication dialogs, ...--